In [ ]:
%reload_ext autoreload
%autoreload 2

#%matplotlib notebook
%matplotlib inline

from math import pi
import numpy as np
import matplotlib.pylab as plt

import gtsam

# 1. Pose2SLAM example

Adopted from `python/gtsam/tests/test_Pose2SLAMExample.py`

In [ ]:
import plot
import audio_stack

graph = gtsam.NonlinearFactorGraph()

# prior
prior_mean = gtsam.Pose2(0.0, 0.0, 0.0)  # prior at origin
prior_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
graph.add(gtsam.PriorFactorPose2(1, prior_mean, prior_noise))

# odometry
odometry_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(
    gtsam.BetweenFactorPose2(1, 2, gtsam.Pose2(2.0, 0.0, 0.0), odometry_noise))
graph.add(
    gtsam.BetweenFactorPose2(2, 3, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometry_noise))
graph.add(
    gtsam.BetweenFactorPose2(3, 4, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometry_noise))
graph.add(
    gtsam.BetweenFactorPose2(4, 5, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometry_noise))

# loop closure
model = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(gtsam.BetweenFactorPose2(5, 2, gtsam.Pose2(2.0, 0.0, pi / 2), model))

# initialization
initial_estimate = gtsam.Values()
initial_estimate.insert(1, gtsam.Pose2(0.5, 0.0, 0.2))
initial_estimate.insert(2, gtsam.Pose2(2.3, 0.1, -0.2))
initial_estimate.insert(3, gtsam.Pose2(4.1, 0.1, pi / 2))
initial_estimate.insert(4, gtsam.Pose2(4.0, 2.0, pi))
initial_estimate.insert(5, gtsam.Pose2(2.1, 2.1, -pi / 2))

# Optimize using Levenberg-Marquardt optimization with an ordering from colamd
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate)
print("Optimizing...")
result = optimizer.optimizeSafely()
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

# plot results
marginals = gtsam.Marginals(graph, result)
plt.figure()
for i in range(1, 6):
    result.atPose2(i)
    #x, y, angle = result.atPose2(i).x(), result.atPose2(i).y(), result.atPose2(
    #    i).theta()
    P = marginals.marginalCovariance(i)
    #plt.scatter(x, y, s=50, label=f'pose {i}')
    #plt.arrow(x, y, np.cos(angle), np.sin(angle), width=0.01)
    plot.plot_pose2(
        1,
        pose=result.atPose2(i),
        covariance=P,
    )
plt.axis('equal')
#plt.legend(loc='upper left')
plt.grid()


# 2. SFM example

Adopted from `http://docs.ros.org/en/melodic/api/gtsam/html/SFMExample_8py_source.html`

In [ ]:
from gtsam import symbol_shorthand
L = symbol_shorthand.L
X = symbol_shorthand.X

from gtsam.examples import SFMdata
from gtsam import (Cal3_S2, DoglegOptimizer,
                         GenericProjectionFactorCal3_S2, Marginals,
                         NonlinearFactorGraph, PinholeCameraCal3_S2, Point3,
                         Pose3, PriorFactorPoint3, PriorFactorPose3, Rot3, Values)

# Define the camera calibration parameters
K = Cal3_S2(50.0, 50.0, 0.0, 50.0, 50.0)

# Define the camera observation noise model
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)  # one pixel in u and v

# Create the set of ground-truth landmarks
points = SFMdata.createPoints()

# Create the set of ground-truth poses
poses = SFMdata.createPoses(K)

# Create a factor graph
graph = NonlinearFactorGraph()

# Add a prior on pose x1. This indirectly specifies where the origin is.
# 0.3 rad std on roll,pitch,yaw and 0.1m on x,y,z
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.3, 0.1, 0.1, 0.1]))
factor = PriorFactorPose3(X(0), poses[0], pose_noise)
graph.push_back(factor)

# Simulated measurements from each camera pose, adding them to the factor graph
for i, pose in enumerate(poses):
    camera = PinholeCameraCal3_S2(pose, K)
    for j, point in enumerate(points):
        measurement = camera.project(point)
        factor = GenericProjectionFactorCal3_S2(
            measurement, measurement_noise, X(i), L(j), K)
        graph.push_back(factor)

# Because the structure-from-motion problem has a scale ambiguity, the problem is still under-constrained
# Here we add a prior on the position of the first landmark. This fixes the scale by indicating the distance
# between the first camera and the first landmark. All other landmark positions are interpreted using this scale.
point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
factor = PriorFactorPoint3(L(0), points[0], point_noise)
graph.push_back(factor)
#graph.print('Factor Graph:\n')

# Create the data structure to hold the initial estimate to the solution
# Intentionally initialize the variables off from the ground truth
initial_estimate = Values()
for i, pose in enumerate(poses):
    transformed_pose = pose.retract(0.1*np.random.randn(6,1))
    initial_estimate.insert(X(i), transformed_pose)
for j, point in enumerate(points):
    transformed_point = point + 0.1*np.random.randn(3)
    initial_estimate.insert(L(j), transformed_point)
#initial_estimate.print('Initial Estimates:\n')

# Optimize the graph and print results
params = gtsam.DoglegParams()
#params.setVerbosity('TERMINATION')
params.setVerbosity('')
optimizer = DoglegOptimizer(graph, initial_estimate, params)
print('Optimizing...')
result = optimizer.optimize()
#result.print('Final results:\n')
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

#print(result)

marginals = Marginals(graph, result)
fig = plt.figure()
plot.plot_3d_points(1, result, marginals=marginals, cov_scale=0.1)
plot.plot_trajectory(1, result, cov_scale=0.1, marginals=marginals, axis_length=8)
plot.set_axes_equal(1)
fig.set_size_inches(10, 10)

# 3. Planar SLAM example

## 3.1 Understand the plane object

In [ ]:
from helpers import get_angles, get_vector, print_plane
from plot import plot_projections
from gtsam import Unit3, Pose2
from gtsam import OrientedPlane3 as Plane

X = symbol_shorthand.X
P = symbol_shorthand.P

estimate = Values()
plane = Plane(n=Unit3(-np.array([0.0, 1.0, 0])), d=1.0) # in local coordinates
estimate.insert(P(0), plane)

# first pose at origin
pose0 = Pose3(r=Rot3.Ypr(0.0, 0.0, 0.0), t=Point3(0.0, 0.0, 0.0))
estimate.insert(X(0), pose0)
print_plane("initial", plane)

pose1 = pose0
for i in np.arange(8):
    # first t, then r!
    pose_delta = Pose3(r=Rot3.Ypr(np.pi/4, 0, 0), t=Point3(0.0, 0.2, 0.0))#, t=Point3(0.0, 0.1, 0.0))
    pose1 = pose1 * pose_delta
    estimate.insert(X(i+1), pose1)
    plane2 = plane.transform(pose1) #, plane)
    print_plane("next", plane2)

plot_projections(estimate)

## 3.2 Simulate measurements

In [ ]:
from helpers import WallSimulation
from gtsam import noiseModel, Sampler

np.random.seed(1)
X = symbol_shorthand.X
P = symbol_shorthand.P

wall_simulation = WallSimulation()
real = Values()
initial_guess = Values()
times = range(1, 4)

pose_0 = Pose3()
plane = Plane(n=Unit3([0, -1, 0]), d=1.0)
print_plane("initial", plane)

real.insert(X(0), pose_0)
real.insert(P(0), plane)

wall_simulation = WallSimulation()
wall_simulation.initialize(plane, pose_0)


#plane_initial = Plane(n=Unit3([1, 1, 1]), d=0.5) # doesn't work!
plane_initial = Plane(n=Unit3([-1, -1, -1]), d=0.5) # works!
initial_guess.insert(X(0), pose_0)
initial_guess.insert(P(0), plane_initial)

for t in times:
    wall_simulation.move_until_time(time=t)
    
    pose_t = wall_simulation.real_pose()
    pose_t_expected = wall_simulation.expected_pose(time=t)
    real.insert(X(t), pose_t)
    initial_guess.insert(X(t), pose_t_expected)
    
    # get plane measurements
    plane_local, distance, azimuth, elevation = wall_simulation.measure_plane()
    print_plane(f" pose {t}", plane_local)
plot_projections(real, perspective=False)
plot.plot_trajectory(1, initial_guess, axis_length=0.2, ls=":")
plot.plot_trajectory(2, initial_guess, axis_length=0.2, ls=":")
plt.show()

## 3.3 Create factor graph

In [ ]:
from gtsam import PriorFactorOrientedPlane3 # is created automatically!
from gtsam import OrientedPlane3Factor as PlaneFactor
from helpers import rad, deg

wall_simulation = WallSimulation()
wall_simulation.initialize(plane, pose_0)

plane_confidence = gtsam.noiseModel.Diagonal.Sigmas([rad(10), rad(10), 5e-2]) # angle, angle, distance
pose_confidence = gtsam.noiseModel.Diagonal.Sigmas([rad(10)]*3 + [1e-2, 1e-2, 1e-2]) # 3 angles, x y z

graph = NonlinearFactorGraph()

for t in times: 
    wall_simulation.move_until_time(t)
    
    # pose measurements
    pose_t = wall_simulation.measure_pose()
    factor = PriorFactorPose3(X(t), pose_t, pose_confidence)
    graph.push_back(factor)
    print(f"Added prior factor to pose {t}: {pose_t.translation().round(3)}")
    
    
    # plane measurements
    plane_meas, azimuth, elevation, distance = wall_simulation.measure_plane()
    
    factor = PlaneFactor(plane_meas.planeCoefficients(), plane_confidence, X(t), P(0))
    graph.push_back(factor)
    print(f"Added plane factor from pose {t} with distance: {plane_meas.distance():.2f}")

In [ ]:
print(graph)

## 3.5 Solve factor graph

In [ ]:
from gtsam import GaussNewtonOptimizer

optimizer = GaussNewtonOptimizer(graph, initial_guess)

print('Optimizing...')
result = optimizer.optimize()

print('initial error = {}'.format(graph.error(initial_guess)))
print('final error = {}'.format(graph.error(result)))

plot_projections(result, perspective=False, ls="--")
plot.plot_trajectory(1, initial_guess, axis_length=0.2, ls=":")
plot.plot_trajectory(2, initial_guess, axis_length=0.2, ls=":")
plot.plot_trajectory(1, real, axis_length=0.2, ls="-")
plot.plot_trajectory(2, real, axis_length=0.2, ls="-")

In [ ]:
marginals = Marginals(graph, result)
print(marginals)

## 3.6 Online inference

### Debugging version

In [ ]:
# below is inspired from gtsam/python/tests/test_VisualISAMExample 
# and python/gtsam/utils/visual_isam.py
from helpers import rad

params = gtsam.ISAM2Params()
params.setRelinearizeSkip(1) # always relinearize

isam = gtsam.ISAM2(params=params)

new_factors = gtsam.NonlinearFactorGraph()
initial_estimates = gtsam.Values()
all_initial_estimates = gtsam.Values()

plane = gtsam.OrientedPlane3(Unit3([0, -1.0, 0]), 1)
pose_0 = gtsam.Pose3()
initial_estimates.insert(P(0), plane)
all_initial_estimates.insert(P(0), plane)

#distance, azimuth, elevation
plane_noise = gtsam.noiseModel.Diagonal.Sigmas([1e-2, rad(1e-10), rad(1e-10)])
pose_noise = gtsam.noiseModel.Diagonal.Sigmas([
    0.01, # x
    0.01, # y 
    1e-10, # z
    rad(1), # roll
    rad(1), # pitch
    rad(3), # yaw
])

isam.update(new_factors, initial_estimates)

wall_simulation = WallSimulation()
wall_simulation.initialize(plane, pose_0)
wall_simulation.set_velocities(linear_m_s=0.1, yaw_deg_s=45)
wall_simulation.plane_noise = plane_noise
wall_simulation.pose_noise = pose_noise

times = np.linspace(0, 4, 5)

for t, time in enumerate(times):
    wall_simulation.move_until_time(time)
    
    pose_prior = wall_simulation.expected_pose()
    pose_t = wall_simulation.measure_pose()
    plane_t = wall_simulation.measure_plane()
    
    factors = gtsam.NonlinearFactorGraph()
    initial_estimates = gtsam.Values()
    
    factor = PriorFactorPose3(X(t), pose_t, pose_noise)
    factors.push_back(factor)
    
    initial_estimates.insert(X(t), pose_prior)
    all_initial_estimates.insert(X(t), pose_prior)
    
    #isam.update(factors, initial_estimates)
    #factors = gtsam.NonlinearFactorGraph()
    #initial_estimates = gtsam.Values()
    
    plane_meas, azimuth, elevation, distance = wall_simulation.measure_plane()
    #print(f"plane factor with {distance, azimuth_deg, elevation_deg}")
    factor = PlaneFactor(plane_meas.planeCoefficients(), plane_noise, X(t), P(0))
    factors.push_back(factor)
    
    isam.update(factors, initial_estimates)
    
    new_result = isam.calculateEstimate()
    
plot_projections(new_result, top=True, side=False, perspective=False)
plt.title('result')
plt.show()
plot_projections(all_initial_estimates, top=True, side=False, perspective=False)
plt.title('initial estimates')
plt.show()

print("plane result", new_result.atOrientedPlane3(P(0)))

### Live version

In [ ]:
import sys
sys.path.append('../../src/audio_gtsam/')
from audio_gtsam.wall_backend import WallBackend

wall_backend = WallBackend()
wall_backend.set_confidence()

plane = gtsam.OrientedPlane3(Unit3([0, -1.0, 0]), 1)
pose_0 = gtsam.Pose3()
wall_simulation = WallSimulation()
wall_simulation.initialize(plane, pose_0)
wall_simulation.set_velocities(linear_m_s=0.1, yaw_deg_s=45)
wall_simulation.plane_noise = wall_backend.plane_noise
wall_simulation.pose_noise = wall_backend.pose_noise

for t, time in enumerate(times):
    wall_simulation.move_until_time(time)
    
    pose_t = wall_simulation.measure_pose()
    #pose_t = wall_simulation.expected_pose()
    
    plane_meas, azimuth, elevation, distance = wall_simulation.measure_plane()
    
    wall_backend.add_pose(r_world=pose_t.translation(), 
                          yaw=pose_t.rotation().yaw())
    plane_factor = wall_backend.add_plane(distance=distance, azimuth=azimuth, elevation=elevation)
    planes, poses = wall_backend.get_results()
    
plot_projections(wall_backend.all_initial_estimates, top=False, side=False, ls=":")
plot_projections(wall_backend.result, top=False, side=False, )
plt.show()

## First attempts to implement custom Planar SLAM factor

In [ ]:
import gtsam
import numpy as np
from typing import List

expected = Rot3.Quaternion(1, 1, 1, 1)

def error_func(this: gtsam.CustomFactor, v: gtsam.Values, H: List[np.ndarray]):
    """
    Error function that mimics a BetweenFactor
    :param this: reference to the current CustomFactor being evaluated
    :param v: Values object
    :param H: list of references to the Jacobian arrays
    :return: the non-linear error
    """
    key0 = this.keys()[0] # plane
    key1 = this.keys()[1] # pose
    rot3, pose3 = v.atRot3(key0), v.atPose3(key1)
    
    # TODO: probably this can be done differently. 
    error = gtsam.Rot3.Logmap(gtsam.Rot3(*pose3.matrix().dot(rot3.quaternion())) * expected)
    if H is not None:
        result = error
        H[0] = np.zeros((3, 3)) # TODO: jacobian with respect to plane
        H[1] = np.zeros((3, 6)) # TODO: jacobian with respect to pose
    return error

noise_model = gtsam.noiseModel.Unit.Create(3)
factor = gtsam.CustomFactor(noise_model, [P(0), X(0)], error_func)

graph = gtsam.NonlinearFactorGraph()
graph.push_back(factor)

initial_estimate = gtsam.Values()
initial_estimate.insert(P(0), rot3)
initial_estimate.insert(X(0), pose3)
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate)
result = optimizer.optimize()
print(result)

In [ ]:
import gtsam
pose3 = gtsam.Pose3()
rot3 = gtsam.Rot3()
rot3_local_quat = pose3.matrix().dot(rot3.quaternion())
rot3_local = gtsam.Rot3(*rot3_local_quat)
rot_error = rot3_local * rot3